In [1]:
!python TransE.py

Epoch 1 | loss: 6.219164
Epoch 2 | loss: 0.000000
Epoch 3 | loss: 0.000000
Epoch 4 | loss: 0.000000
Epoch 5 | loss: 0.000000
Epoch 6 | loss: 0.000000
Epoch 7 | loss: 0.000000
Epoch 8 | loss: 0.000000
Epoch 9 | loss: 0.000000
Epoch 10 | loss: 0.000000
Epoch 11 | loss: 0.000000
Epoch 12 | loss: 0.000000
Epoch 13 | loss: 0.000000
Epoch 14 | loss: 0.000000
Epoch 15 | loss: 0.000000
Epoch 16 | loss: 0.000000
Epoch 17 | loss: 0.012650
Epoch 18 | loss: 0.000000
Epoch 19 | loss: 0.000094
Epoch 20 | loss: 0.015199
Epoch 21 | loss: 0.000000
Epoch 22 | loss: 0.000000
Epoch 23 | loss: 0.000000
Epoch 24 | loss: 0.000000
Epoch 25 | loss: 0.000000
Epoch 26 | loss: 0.000000
Epoch 27 | loss: 0.000000
Epoch 28 | loss: 0.000000
Epoch 29 | loss: 0.000000
Epoch 30 | loss: 0.000000
Epoch 31 | loss: 0.000000
Epoch 32 | loss: 0.000000
Epoch 33 | loss: 0.000000
Epoch 34 | loss: 0.000000
Epoch 35 | loss: 0.000000
Epoch 36 | loss: 0.000000
Epoch 37 | loss: 0.000000
Epoch 38 | loss: 0.000000
Epoch 39 | loss: 0.00